<a href="https://colab.research.google.com/github/rickqiu/trax_chatbots/blob/main/chatbots_conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Chatbots Conversation
@author: Rick Qiu

This notebook demonstrates examples of conversations between two chatbots, who have local knowledge in multiple domains such as attraction, hospital, hotel, police, taxi and train. 

The demo uses Google Trax deep learning library and a pre-trained Reformer language model "chatbot_model1.pkl.gz" from COURSERA NLP Specialization. 

Here is a list of useful resources.


- [MultiWoz](https://arxiv.org/abs/1810.00278) dataset
- [Reformer](https://arxiv.org/abs/2001.04451) paper
- [Trax](https://github.com/google/trax) code repository
- COURSERA [NLP Specialization](https://www.coursera.org/specializations/natural-language-processing)

### Acknowledgements
Many thanks to Łukasz Kaiser and his Google Trax open source project team members

Special thanks to NLP Specialization instructor Younes Bensouda Mourri and all staff






## 1. Setup

In [1]:
!pip install -q -U https://storage.googleapis.com/jax-releases/cuda101/jaxlib-0.1.55-cp36-none-manylinux2010_x86_64.whl
!pip install -q -U jax
!pip install -q -U trax

     |████████████████████████████████| 144.8MB 21kB/s 
     |████████████████████████████████| 481kB 7.9MB/s 
     |████████████████████████████████| 419kB 6.3MB/s 
     |████████████████████████████████| 1.5MB 12.8MB/s 
     |████████████████████████████████| 174kB 47.7MB/s 
     |████████████████████████████████| 2.6MB 53.5MB/s 
     |████████████████████████████████| 5.3MB 60.3MB/s 
     |████████████████████████████████| 194kB 67.9MB/s 
     |████████████████████████████████| 368kB 65.8MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 348kB 70.9MB/s 
     |████████████████████████████████| 655kB 64.0MB/s 
     |████████████████████████████████| 358kB 52.3MB/s 
     |████████████████████████████████| 983kB 60.6MB/s 
     |████████████████████████████████| 1.1MB 62.4MB/s 
     |████████████████████████████████| 3.6MB 65.7MB/s 
     |████████████████████████████████| 1.1MB 70.8MB/s 
     |████████████████████████████████| 71kB 9.3MB/

In [2]:
!pip list | grep trax

trax                          1.3.5                


In [3]:
# clone to get vocabs and the pretrained model parts from repository
!git clone https://github.com/rickqiu/trax_chatbots.git

Cloning into 'trax_chatbots'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 75 (delta 34), reused 0 (delta 0), pack-reused 17
Unpacking objects: 100% (75/75), done.
Checking out files: 100% (11/11), done.


In [4]:
%cd trax_chatbots
!tar -xzvf vocabs.tar.gz
!cat model_splits/* > chatbot_model1.pkl.gz
%cd ..

/content/trax_chatbots
vocabs/
vocabs/en_32k.sentencepiece
vocabs/en_32k.sentencepiece.vocab
vocabs/en_32k.subword
vocabs/en_8k.subword
/content


In [5]:
# import libraries
import numpy as np
import trax   
from trax import layers as tl

## 2. Modelling

In [16]:
# define attention for fast inference
def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 200
    kwargs['predict_drop_len'] = 128
    return tl.SelfAttention(*args, **kwargs)

# define the model
model = trax.models.reformer.ReformerLM( 
        vocab_size=33000,
        n_layers=6,
        mode='predict',
        attention_type=attention
    )

In [17]:
# display the Reformer model
print(str(model))

Serial[
  ShiftRight(1)
  Embedding_33000_512
  Dropout
  PositionalEncoding
  Dup_out2
  ReversibleSerial_in2_out2[
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        FastGelu
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        FastGelu
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        

## 3.  Prediction

In [18]:
# define an input signature
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)

# initialize the model from file
model.init_from_file('trax_chatbots/chatbot_model1.pkl.gz',
                     weights_only=True, input_signature=shape11)

# save the starting state
STARTING_STATE = model.state

In [19]:
# https://www.deeplearning.ai/natural-language-processing-specialization/
# vocabulary file directory
VOCAB_DIR = './trax_chatbots/vocabs'

# vocabulary filename
VOCAB_FILE = 'en_32k.subword'

def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]


def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)


def generate_output(model, start_sentence, vocab_file, vocab_dir, temperature):
    """
    Args:
        model:  the Reformer language model you just trained
        start_sentence (string): starting sentence of the conversation
        vocab_file (string): vocabulary filename
        vocab_dir (string): directory of the vocabulary file
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        generator: yields the next symbol generated by the model
    """
    
    input_tokens =  tokenize(start_sentence, vocab_file, vocab_dir)
    
    # add batch dimension to array
    input_tokens_with_batch = np.expand_dims(input_tokens, axis=0)
    
    # call the autoregressive_sample_stream function from trax
    output_gen = trax.supervised.decoding.autoregressive_sample_stream( 
        model,
        inputs=input_tokens_with_batch,
        temperature=temperature
    )
    
    return output_gen


def generate_dialogue(model, model_state, start_sentence, vocab_file, vocab_dir, max_len, temperature):
    """
    Args:
        model:  the Reformer language model you just trained
        model_state (np.array): initial state of the model before decoding
        start_sentence (string): starting sentence of the conversation
        vocab_file (string): vocabulary filename
        vocab_dir (string): directory of the vocabulary file
        max_len (int): maximum number of tokens to generate 
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        generator: yields the next symbol generated by the model
    """  
    
    # define the delimiters we used during training
    delimiter_1 = 'Person 1: ' 
    delimiter_2 = 'Person 2: '
    
    # initialize detokenized output
    sentence = ''
    
    # token counter
    counter = 0

    # turns
    turns = 0
    
    # output tokens
    result = []
    
    # reset the model state when starting a new dialogue
    model.state = model_state
    
    # calls the output generator implemented earlier
    output = generate_output(model, start_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=temperature)
    
    # print the starting sentence
    print(start_sentence.split(delimiter_2)[0].strip())

    # turns
    turns = 1
    
    # loop below yields the next tokens until max_len is reached. the if-elif is just for prettifying the output.
    for o in output:
        
        result.append(o)
        
        sentence = detokenize(np.concatenate(result, axis=0), vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)
        
        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(f'{delimiter_2}{sentence}')
            sentence = ''
            result.clear()
            turns += 1
        
        elif sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            print(f'{delimiter_1}{sentence}')
            sentence = ''
            result.clear()
            turns +=1

        counter += 1
        
        if counter > max_len and turns%2 == 0:
           break    

## 4. Outputing results

In [30]:
sample_sentence = ' Person 1: Is there a hospital nearby? Person 2: '
generate_dialogue(model, model_state=STARTING_STATE, 
                  start_sentence=sample_sentence,
                  vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, 
                  max_len=100, temperature=0.2)

Person 1: Is there a hospital nearby?
Person 2: Addensbrookes Hospital is located at Hills Rd, Cambridge, postcode CB20QQ. Do you need anything else? 
Person 1: No, that's all. Thank you. 
Person 2: You're welcome. Have a good day.m.Thanks for contacting the Cambridge TownInfo centre. Goodbye.
Person 1: Thank you for your help. 
Person 2: Thank you for using our services.Goodbye.


In [32]:
sample_sentence = ' Person 1: Can you book a taxi? Person 2: '
generate_dialogue(model, model_state=STARTING_STATE, 
                  start_sentence=sample_sentence, 
                  vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, 
                  max_len=100, temperature=0.2)

Person 1: Can you book a taxi?
Person 2: I sure can. Where will you be going? 
Person 1: I'm going to be from the city stop restaurant. 
Person 2: Okay, I have booked you a taxi. It's a yellow ford and the contact number is 07827592182. Is there anything else I can help you with? 
Person 1: No, that's all I need. Thank you. 
Person 2: Thank you for using our services.Goodbye.


In [36]:
sample_sentence = ' Person 1: Are there theatres in town? Person 2: '
generate_dialogue(model, model_state=STARTING_STATE, 
                  start_sentence=sample_sentence, 
                  vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, 
                  max_len=240, temperature=0.15)

Person 1: Are there theatres in town?
Person 2: There are 4 theatres in town. Do you have a preference? 
Person 1: No, I don't have a preference. Which one do you recommend? 
Person 2: I would recommend the Mumford Theatre. It is located at Anglia Ruskin Enterprise, east road. Would you like more information on it? 
Person 1: Yes, I would like to know the entrance fee, and postcode please. 
Person 2: The postcode is cb11pt. Can I help you with anything else? 
Person 1: I'm looking for a train that leaves on Saturday from Bishops Stortford. 
Person 2: There are 10 trains that meet your criteria. Do you have a specific time you would like to arrive by? 
Person 1: No, but I would like to arrive by 10:45. 
Person 2: TR0635 arrives at 10:07. Would you like to book a seat? 
Person 1: Yes, I'll need 7 tickets please. 
Person 2: Booking was successful, the total fee is 70.7 GBP payable at the station .
Reference number is : P1GSGSMAL2. 
